In [1]:
import pandas as pd
import glob

/home/jaeyeonl/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid", palette="pastel")


In [3]:
aime24df = pd.read_parquet("hf://datasets/Maxwell-Jia/AIME_2024/aime_2024_problems.parquet")
aime24df = aime24df.rename(columns={'Problem': 'input', 'Answer': 'target'})
aime24df = aime24df[['input', 'target']]

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = aime24df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


aime_dfs = []
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-msprt-1e61e6005094994-3/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-msprt-1e61e6005094994/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-msprt-1e61e6005094994/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-high")]

pd.concat(aime_dfs)

,mean_trials,accuracy,method,benchmark
0,40.000000,0.700000,Self-Cons@40,AIME24 / o3-mini-low
0,16.400000,0.700000,p-value,AIME24 / o3-mini-low
0,16.266667,0.700000,Ada-Cons,AIME24 / o3-mini-low
0,11.466667,0.800000,SPRT (Ours),AIME24 / o3-mini-low
0,13.866667,0.766667,mSPRT (Ours),AIME24 / o3-mini-low
0,40.000000,0.900000,Self-Cons@40,AIME24 / o3-mini-medium
0,11.200000,0.933333,p-value,AIME24 / o3-mini-medium
0,10.366667,0.900000,Ada-Cons,AIME24 / o3-mini-medium
0,4.733333,0.900000,SPRT (Ours),AIME24 / o3-mini-medium
0,6.500000,0.900000,mSPRT (Ours),AIME24 / o3-mini-medium


In [4]:
import random
SEED = 42
random.seed(SEED)

df = pd.read_csv("../../data/gpqa_diamond.csv")

answer_columns = [
    "Correct Answer",
    "Incorrect Answer 1",
    "Incorrect Answer 2",
    "Incorrect Answer 3"
]

options_labels = ["A", "B", "C", "D"]

formatted_questions = []
correct_answers = []

for _, row in df.iterrows():
    shuffled_columns = random.sample(answer_columns, len(answer_columns))

    shuffled_options = {opt: row[col] for opt, col in zip(options_labels, shuffled_columns)}

    question_text = f"Question: {row['Pre-Revision Question']}\\n"
    choices_text = "\\n".join([f"{opt}. {shuffled_options[opt]}" for opt in options_labels])
    prompt_text = f"{question_text}{choices_text}"

    correct_option = [opt for opt, col in zip(options_labels, shuffled_columns) if col == "Correct Answer"][0]

    formatted_questions.append(prompt_text)
    correct_answers.append(correct_option)

gpqa_df = pd.DataFrame({
    "input": formatted_questions,
    "target": correct_answers
})
gpqa_df

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = gpqa_df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


gpqa_dfs = []
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_40_trail_multi_choice/*")).T.assign(method="Self-Cons@40", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_bayesian_posterior_multi_choice/*")).T.assign(method="Ada-Cons", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_msprt_multi_choice/*")).T.assign(method="mSPRT (Ours)", benchmark="GPQA Diamond / o3-mini-low")]
pd.concat(gpqa_dfs)


,mean_trials,accuracy,method,benchmark
0,40.000000,0.681818,Self-Cons@40,GPQA Diamond / o3-mini-low
0,8.878788,0.671717,Ada-Cons,GPQA Diamond / o3-mini-low
0,4.520202,0.686869,mSPRT (Ours),GPQA Diamond / o3-mini-low


In [ ]:
gsm_df = pd.read_json('../../data/gsm.jsonl', lines=True)

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = gsm_df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


gsm_dfs = []
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-vote/*")).T.assign(method="Self-Cons@40", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-bayesian/*")).T.assign(method="Ada-Cons", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-sprt-0510050947/*")).T.assign(method="SPRT (Ours)", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-msprt-25600509446/*")).T.assign(method="mSPRT (Ours)", benchmark="GSM / gpt-4o-mini")]
pd.concat(gsm_dfs)


In [ ]:
df2 = pd.concat(aime_dfs + gpqa_dfs + gsm_dfs).pivot_table(values=["accuracy", "mean_trials"], columns="method", index="benchmark", sort=False)
df2 = df2.rename(columns={"accuracy": "Accuracy", "mean_trials": "Num. Trials"})
df2.index = df2.index.set_names("Benchmark and LLM")
df2.columns = df2.columns.set_names([None, "Method"])
df2

Num. Trials                                    \
Method                     Self-Cons@40    p-value   Ada-Cons SPRT (Ours)   
Benchmark and LLM                                                           
AIME24 / o3-mini-low               40.0  16.400000  16.266667   11.466667   
AIME24 / o3-mini-medium            40.0  11.200000  10.366667    4.733333   
AIME24 / o3-mini-high              40.0   9.733333   7.400000    4.133333   
GPQA Diamond / o3-mini-low         40.0        NaN   8.878788         NaN   
GSM / gpt-4o-mini                  40.0        NaN   6.399545    4.241092   

                                            Accuracy                      \
Method                     mSPRT (Ours) Self-Cons@40   p-value  Ada-Cons   
Benchmark and LLM                                                          
AIME24 / o3-mini-low          11.366667     0.700000  0.700000  0.700000   
AIME24 / o3-mini-medium        6.500000     0.900000  0.933333  0.900000   
AIME24 / o3-mini-high          4.366667     0.966667  0.966667  0.966667   
GPQA Diamond / o3-mini-low     4.520202     0.681818       NaN  0.671717   
GSM / gpt-4o-mini              4.123578     0.941622       NaN  0.941622   

                                                     
Method                     SPRT (Ours) mSPRT (Ours)  
Benchmark and LLM                                    
AIME24 / o3-mini-low          0.800000     0.633333  
AIME24 / o3-mini-medium       0.900000     0.900000  
AIME24 / o3-mini-high         0.966667     0.966667  
GPQA Diamond / o3-mini-low         NaN     0.686869  
GSM / gpt-4o-mini             0.937832     0.936315

In [ ]:
print((df2["Accuracy"]*100).to_latex(float_format="%.1f\\%%"))
print(df2["Num. Trials"].to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
Method & Self-Cons@40 & p-value & Ada-Cons & SPRT (Ours) & mSPRT (Ours) \\
Benchmark and LLM &  &  &  &  &  \\
\midrule
AIME24 / o3-mini-low & 70.0\% & 70.0\% & 70.0\% & 80.0\% & 63.3\% \\
AIME24 / o3-mini-medium & 90.0\% & 93.3\% & 90.0\% & 90.0\% & 90.0\% \\
AIME24 / o3-mini-high & 96.7\% & 96.7\% & 96.7\% & 96.7\% & 96.7\% \\
GPQA Diamond / o3-mini-low & 68.2\% & NaN & 67.2\% & NaN & 68.7\% \\
GSM / gpt-4o-mini & 94.2\% & NaN & 94.2\% & 93.8\% & 93.6\% \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
Method & Self-Cons@40 & p-value & Ada-Cons & SPRT (Ours) & mSPRT (Ours) \\
Benchmark and LLM &  &  &  &  &  \\
\midrule
AIME24 / o3-mini-low & 40.00 & 16.40 & 16.27 & 11.47 & 11.37 \\
AIME24 / o3-mini-medium & 40.00 & 11.20 & 10.37 & 4.73 & 6.50 \\
AIME24 / o3-mini-high & 40.00 & 9.73 & 7.40 & 4.13 & 4.37 \\
GPQA Diamond / o3-mini-low & 40.00 & NaN & 8.88 & NaN & 4.52 \\
GSM / gpt-4o-mini & 40.00 & NaN & 6.40 & 4.24 & 4.12 \\
\bottomrule
\e

> **Note**: Copy the above result and put \bf to the remark our result.